In [ ]:
'''
Multidimensional Retail Data Centralization
-------
This script extracts data, cleans and uploads it all to designated tables in sales_data database.
Methods have been predefined so variables are set to organize the extraction, cleaning and uploading process for each set of data.

Contents
-------
    * Imports 
    * Data Extraction Details - assigning urls/links, endpoints and api-keys necessary to run the code.
    * Classes Usage - instance created and initialized and variables assigned to each class.

    The listed cell blocks perform the following for each dataset: Data Extraction, Transformation and Loading to sales_data.
    * Legacy Users
    * Card Data
    * Business Store Data
    * Products Data
    * Orders Table
    * Sales Date Times

'''

In [ ]:
#Imports
from database_utils import DatabaseConnector
from data_cleaning import DataCleaning
from data_extraction import DataExtractor
from data_processing import DataProcessor

In [ ]:
#Data Extraction Details
creds_path = '/Users/itsanya/AiCore/MRDC/db_creds.yaml'

pdf_link = 'https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf'

b_store_api_key = 'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'
number_of_stores_endpoint = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores'
store_endpoint = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/{store_number}'

s3_address = 's3://data-handling-public/products.csv' 

s3_url = 'https://data-handling-public.s3.eu-west-1.amazonaws.com/date_details.json'

In [ ]:
#Classes Usage
db_connector_instance = DatabaseConnector(creds_path) 
data_extractor = DataExtractor(db_connector_instance) 
data_cleaner = DataCleaning()
data_processor = DataProcessor()

In [ ]:
#Legacy Users
#Extract
table_name = 'legacy_users'
users_df = data_extractor.read_rds_table(table_name)

#Clean
fully_cleaned_users_df = data_cleaner._clean_users_df(users_df)

#Upload to dim_users
db_connector_instance._upload_to_db(df=fully_cleaned_users_df, table_name='dim_users')

In [ ]:
#Card Data
#Extract
card_df = data_extractor.retrieve_pdf_data()

#Clean
fully_cleaned_card_df = data_cleaner._clean_card_data(card_df)

#Upload to dim_card_details
db_connector_instance._upload_to_db(df=fully_cleaned_card_df, table_name='dim_card_details')

In [ ]:
#Business Store Data
#Extract
data_extractor.set_api_key(b_store_api_key)
number_of_stores = data_extractor.list_number_of_stores(number_of_stores_endpoint)
b_store_df = data_extractor.retrieve_stores_data(store_endpoint, number_of_stores)

#Clean
fully_cleaned_b_store_df = data_cleaner._clean_b_store_data(b_store_df)

#Upload to dim_store_details
db_connector_instance._upload_to_db(df=fully_cleaned_b_store_df, table_name='dim_store_details')

In [ ]:
#Products Data
#Extract
prods_df = data_extractor.extract_from_s3(s3_address)

#Clean
cleaned_prod_w = data_cleaner._convert_product_weights(prods_df)

fully_cleaned_prods_df = data_cleaner._clean_products_data(cleaned_prod_w)

#Upload to dim_store_details
db_connector_instance._upload_to_db(df=fully_cleaned_prods_df, table_name='dim_products')

In [ ]:
#Orders Table
#Extract
table_name = 'orders_table'
orders_df = data_extractor.read_rds_table(table_name)

#Clean
fully_cleaned_orders_df = data_cleaner._clean_orders_df(orders_df)

#Upload to orders_table
db_connector_instance._upload_to_db(df=fully_cleaned_orders_df, table_name='orders_table')

In [ ]:
#Sales Date Times
#Extract
sdt_df = data_extractor.extract_sdt(s3_url)

#Clean
fully_cleaned_sdt_df = data_cleaner._clean_sdt_df(sdt_df)

#Upload to dim_date_times
db_connector_instance._upload_to_db(df=fully_cleaned_sdt_df, table_name='dim_date_times')